In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install segmentation-models-pytorch
!pip install rasterio
!pip install richdem

     |████████████████████████████████| 88 kB 4.0 MB/s 
     |████████████████████████████████| 376 kB 30.2 MB/s 
     |████████████████████████████████| 58 kB 7.1 MB/s 
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.6.3-py3-none-any.whl size=12421 sha256=9acdcf362d28de67ffa1dc1ec78b8477b5c5c11fff1ce7a457b53303281a5277
  Stored in directory: /root/.cache/pip/wheels/90/6b/0c/f0ad36d00310e65390b0d4c9218ae6250ac579c92540c9097a
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60965 sha256=abe3ffbd17c5d4da9e98fcd5dd092ce162040a31bd48da2424af79dc33a204a6
  Stored in directory: /root/.cache/pip/wheels/ed/27/e8/9543d42de2740d3544db96aefef63bda3f2c1761b3334f4873
Successfully built efficientnet-pytorch pretrainedmodels
     |████████████████████████████████| 19.3 MB 1.2 MB/s 
     |████████████████████████████████| 4.5 MB 5.1 MB/s 


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import rasterio
from rasterio.plot import show
from pathlib import Path
import shutil
import pandas as pd  
import richdem as rd

In [ ]:
# Root Directory
Image_Segmentation_Path = '/content/drive/My Drive/Image Segmentation/'

# Inputs/Sources

dataframes_path = Image_Segmentation_Path + "DataFrames/"

# This is the entire dataset. We'll do it later.
# dirs_pairs = dataframes_path + "complete_dirs_pairs.csv"
# train_dirs_path = dataframes_path + 'complete_train_dirs.csv'
# validate_dirs_path = dataframes_path + 'complete_validate_dirs.csv'
# test_dirs_path = dataframes_path + 'complete_test_dris.csv'


dirs_pairs = dataframes_path + "origin_dirs_pairs.csv"
train_dirs_path = dataframes_path + 'origin_train_dirs.csv'
validate_dirs_path = dataframes_path + 'origin_validate_dirs.csv'
test_dirs_path = dataframes_path + 'origin_test_dris.csv'


#dirs_pairs = dataframes_path + "slope_dirs_pairs.csv"
#train_dirs_path = dataframes_path + 'slope_train_dirs.csv'
#validate_dirs_path = dataframes_path + 'slope_validate_dirs.csv'
#test_dirs_path = dataframes_path + 'slope_test_dris.csv'


#dirs_pairs = dataframes_path + "aspect_dirs_pairs.csv"
#train_dirs_path = dataframes_path + 'aspect_train_dirs.csv'
#validate_dirs_path = dataframes_path + 'aspect_validate_dirs.csv'
#test_dirs_path = dataframes_path + 'aspect_test_dris.csv'


#dirs_pairs = dataframes_path + "hillshade_dirs_pairs.csv"
#train_dirs_path = dataframes_path + 'hillshade_train_dirs.csv'
#validate_dirs_path = dataframes_path + 'hillshade_validate_dirs.csv'
#test_dirs_path = dataframes_path + 'hillshade_test_dris.csv'


In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [ ]:
from torch.utils.data import DataLoader
from torch.utils.data import Dataset as BaseDataset
from torch import FloatTensor

In [ ]:
class Dataset(BaseDataset):
  """
  Args:
      images_dir (str): path to images folder
      masks_dir (str): path to segmentation masks folder
      class_values (list): values of classes to extract from segmentation mask
  """
    
  CLASSES = ['non-mound', 'mound', 'invalid data']
    
  def __init__(
          self, 
          dirs,
          classes=None,
          augmentation=None):
    self.df = pd.read_csv(dirs)
    self.ids = self.df['Index'].values.tolist()
    
    # convert str names to class values on masks
    self.class_values = [0, 1, -1]
      
      
    self.augmentation = augmentation
    
  def __getitem__(self, i):
    # read data
    with rasterio.open(self.df.at[i, 'Input']) as dem:
      dem_array = dem.read(1)
      
    with rasterio.open(self.df.at[i, 'Target']) as mask:
      mask_array = mask.read(1)

    # apply augmentations
    if self.augmentation:
      sample = self.augmentation(image=dem_array, mask=mask_array)
      dem_array, mask_array = sample['image'], sample['mask']
    
    dem_array = scaler.fit_transform(dem_array)
    mask_array = scaler.fit_transform(mask_array)
    
    dem_array = FloatTensor(np.expand_dims(dem_array,axis=(0)))
    mask_array = FloatTensor(np.expand_dims(mask_array,axis=(0)))
    
    return dem_array, mask_array
      
  def __len__(self):
    return len(self.ids)

In [ ]:
dataset = Dataset(dirs_pairs, classes=['mound'])
dem_array, mask_array = dataset[1] # get some sample

In [ ]:
mask_array.shape

torch.Size([1, 201, 177])

In [ ]:
import albumentations as albu

In [ ]:
def get_training_augmentation():
  train_transform = [albu.PadIfNeeded(min_height=320, min_width=320, always_apply=True, border_mode=0)]

  return albu.Compose(train_transform)


In [ ]:
def get_validation_augmentation():
    """Add paddings to make image shape divisible by 32"""
    test_transform = [
        albu.PadIfNeeded(384, 480)
    ]
    return albu.Compose(test_transform)

In [ ]:
import torch
import numpy as np
import segmentation_models_pytorch as smp

In [ ]:
ENCODER = 'se_resnext50_32x4d'
ENCODER_WEIGHTS = 'imagenet'
ACTIVATION = 'softmax2d' # could be None for logits or 'softmax2d' for multiclass segmentation
CLASSES = [0, 1, -1]
# create segmentation model with pretrained encoder
model = smp.FPN(
    encoder_name=ENCODER, 
    encoder_weights=ENCODER_WEIGHTS, 
    classes=len(CLASSES), 
    in_channels=1,
    activation=ACTIVATION,
)

Downloading: "http://data.lip6.fr/cadene/pretrainedmodels/se_resnext50_32x4d-a260b3a4.pth" to /root/.cache/torch/hub/checkpoints/se_resnext50_32x4d-a260b3a4.pth


  0%|          | 0.00/105M [00:00<?, ?B/s]

In [ ]:
# CLASSES = [0, 1, -1]
# model = smp.Unet(
#     encoder_name="resnet34",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
#     encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
#     classes=len(CLASSES),
#     in_channels=1,
#     activation='softmax2d'                  # model output channels (number of classes in your dataset)
# )

In [ ]:
train_dataset = Dataset(train_dirs_path,
                        augmentation=get_training_augmentation(),                         
                        classes=CLASSES)


valid_dataset = Dataset(validate_dirs_path,
                        augmentation=get_validation_augmentation(),                         
                        classes=CLASSES)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=2)
valid_loader = DataLoader(valid_dataset, batch_size=1, shuffle=False, num_workers=2)

In [ ]:
dem, mask = train_dataset[6]
dem.shape

torch.Size([1, 320, 320])

In [ ]:
torch.max(dem)

tensor(1.)

In [ ]:
# Dice/F1 score - https://en.wikipedia.org/wiki/S%C3%B8rensen%E2%80%93Dice_coefficient
# IoU/Jaccard score - https://en.wikipedia.org/wiki/Jaccard_index

loss = smp.utils.losses.DiceLoss()
metrics = [
    smp.utils.metrics.IoU(threshold=0.5),
]

optimizer = torch.optim.Adam([ 
    dict(params=model.parameters(), lr=0.0001),
])

In [ ]:
# create epoch runners 
# it is a simple loop of iterating over dataloader`s samples
train_epoch = smp.utils.train.TrainEpoch(
    model, 
    loss=loss, 
    metrics=metrics, 
    optimizer=optimizer,
    verbose=True,
)

valid_epoch = smp.utils.train.ValidEpoch(
    model, 
    loss=loss, 
    metrics=metrics, 
    verbose=True,
)

In [ ]:
# train model for 40 epochs

max_score = 0

for i in range(0, 40):
    
    print('\nEpoch: {}'.format(i))
    train_logs = train_epoch.run(train_loader)
    valid_logs = valid_epoch.run(valid_loader)
    
    # do something (save model, change lr, etc.)
    if max_score < valid_logs['iou_score']:
        max_score = valid_logs['iou_score']
        torch.save(model, './best_model.pth')
        print('Model saved!')
        
    if i == 25:
        optimizer.param_groups[0]['lr'] = 1e-5
        print('Decrease decoder learning rate to 1e-5!')


Epoch: 0
valid: 100%|██████████| 546/546 [08:12<00:00,  1.11it/s, dice_loss - 0.9143, iou_score - 0.05764]
Model saved!

Epoch: 1
valid: 100%|██████████| 546/546 [08:14<00:00,  1.10it/s, dice_loss - 0.9143, iou_score - 0.05764]

Epoch: 2
valid: 100%|██████████| 546/546 [08:12<00:00,  1.11it/s, dice_loss - 0.9143, iou_score - 0.05764]

Epoch: 3
train:  72%|███████▏  | 148/205 [32:58<12:37, 13.29s/it, dice_loss - 0.8757, iou_score - 0.06738]

In [ ]:
# load best saved checkpoint
best_model = torch.load('./best_model.pth')

In [ ]:
# create test dataset
test_dataset = Dataset(test_dirs_path,
                       augmentation=get_validation_augmentation(), 
                       preprocessing=get_preprocessing(preprocessing_fn),
                       classes=CLASSES)

test_dataloader = DataLoader(test_dataset)

In [ ]:
# evaluate model on test set
test_epoch = smp.utils.train.ValidEpoch(
    model=best_model,
    loss=loss,
    metrics=metrics)

logs = test_epoch.run(test_dataloader)

In [ ]:

# for i in range(5):
#     n = np.random.choice(len(test_dataset))
    
#     image_vis = test_dataset[n][0].astype('uint8')
#     image, gt_mask = test_dataset[n]
    
    
#     x_tensor = torch.from_numpy(image).unsqueeze(0)
#     pr_mask = best_model.predict(x_tensor)
#     pr_mask = (pr_mask.cpu().numpy().round())
        
#     visualize(
#         image=image_vis, 
#         ground_truth_mask=gt_mask, 
#         predicted_mask=pr_mask
#     )